In [ ]:
import os
import numpy as np
from skimage import io
from matplotlib import pyplot as plt
import cv2
from skimage import color
from scipy.ndimage import binary_fill_holes
from skimage import exposure
from tifffile import imread, imwrite

In [ ]:
def scaled(img, scale=32000):
    """Stretch values out to scale
    Used to be 45000, but changing it down to 32000 as of 7 Aug 2024
    """

    epsilon = 0.999    
    _max = np.quantile(img[img>0], epsilon)
    #_max = np.max(img)
    scaled = (img * (scale / _max)).astype(img.dtype) # scale the image from original values to a broader range of values
    del img
    return scaled, _max


def rescaler(img, lower, upper):
    # Contrast stretching
    plower, pupper = np.percentile(img, (0, 99))
    #_max = np.max(img)
    img_rescale = exposure.rescale_intensity(img, in_range=(plower, pupper))
    #img_rescale = exposure.rescale_intensity(img, in_range='image', out_range='dtype')
    return img_rescale


def equalized(fixed, cliplimit=2):    
    clahe = cv2.createCLAHE(clipLimit=cliplimit, tileGridSize=(8, 8))
    fixed = clahe.apply(fixed)
    return fixed

def clean(img, mask):
    return cv2.bitwise_and(img, img, mask=mask)


In [ ]:
animal = 'DK161'
file = '046.tif'
BASE_DIR = f'/net/birdstore/Active_Atlas_Data/data_root/pipeline_data/{animal}/preps'
filepath = os.path.join(BASE_DIR, 'C2/full', file)
maskpath = os.path.join(BASE_DIR, 'masks/C1/full_masked', file)

In [ ]:
#For bigarr, look at 15812x43685, 16816x44463
if file == '046.tif':
    startrow = 31000
    endrow = 33000
    startcol = 8000
    endcol = 12000
if file == '057.tif':
    startrow = 37000
    endrow = 38250
    startcol = 10700
    endcol = 12000

In [ ]:
img = io.imread(filepath)
print(f'dtype={img.dtype} shape={img.shape}')
mask = io.imread(maskpath)
print(f'dtype={mask.dtype} shape={mask.shape}')

In [ ]:
cleaned = clean(img, mask)
del img
del mask
print(f'dtype={cleaned.dtype} shape={cleaned.shape}')

In [ ]:
%%time
lower = 0
upper = 32000
epsilon = 0.999
cleaned_scaled, _max = scaled(cleaned, upper)
#cleaned_scaled = rescaler(cleaned, lower, upper)
#cleaned_scaled = histogram_equalize(img, mask)
print(f'cleaned_scaled dtype={cleaned_scaled.dtype}')
cropped = cleaned_scaled[startrow:endrow, startcol:endcol]
plt.title(_max)
plt.imshow(cropped, cmap="gray")

In [ ]:
#outpath = os.path.join(BASE_DIR, f'rescaler_upper_{upper}_lower_{lower}_{file}')
outpath = os.path.join(BASE_DIR, f'scaled_upper_{upper}_max_{_max}_{file}')
print(outpath)
imwrite(outpath, cleaned_scaled, compression='LZW', bigtiff=True)

In [ ]:
%%time
shapeto = cleaned.shape
reference = midallenarr[midallenarr > 0]
reference = reference.flatten()
target = cleaned.flatten()
matched = exposure.match_histograms(target, reference)

In [ ]:
%%time
matched = match_histograms(cleaned, midallenarr)
print(f'matched dtype={matched.dtype} shape={matched.shape}')

In [ ]:
#matched = matched.reshape(shapeto)
print(f'matched dtype={matched.dtype}')
cropped = matched[startrow:endrow, startcol:endcol]
plt.title('matched')
plt.imshow(cropped, cmap="gray")

In [ ]:
data = matched / np.max(matched) # normalize the data to 0 - 1
data = 65535 * data # Now scale by 255
outpath = os.path.join(BASE_DIR, f'matched_{file}')
print(outpath)
imwrite(outpath, data.astype(np.uint16), compression='LZW', bigtiff=True)

In [ ]:
img = midallenarr.copy()
img = img[img > 0]
flat = img.flatten()
del img
fig = plt.figure()
plt.rcParams["figure.figsize"] = [10, 6]
end = np.max(flat)
end = 2000
plt.hist(flat, flat.max(), [0, 5000], color='b')
plt.style.use("ggplot")
plt.grid(axis="y", alpha=0.75)
plt.xlabel("Value")
plt.ylabel("Frequency")
plt.show()

In [ ]:
newimg = clean(img, mask)
white = np.where(mask==255)
whiterows = white[0]
firstrow = whiterows[1]
print(firstrow)
print(np.max(newimg[firstrow]))

print(f'dtype={newimg.dtype} shape={newimg.shape} first={first}')
newimg[newimg[:,:,0] == 0] = 234
plt.imshow(newimg)

In [ ]:
scale = 45000
cropped_scaled = scaled(cropped, scale)
print(f'dtype={cropped_scaled.dtype} shape={cropped_scaled.shape}')

In [ ]:
plt.imshow(cropped_scaled, cmap="gray")

In [ ]:
outpath = os.path.join(BASE_DIR, 'cropped_test.tif')
io.imsave(outpath, cropped_scaled)

In [ ]:
scale = 2**16 - 1
cropped_scaled = scaled(cropped, scale)
outpath = os.path.join(DIR, f'cropped_scaled_{scale}.tif')
if not os.path.exists(outpath):
    io.imsave(outpath, cropped_scaled)